In [1]:
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install tqdm ipywidgets
#%pip install gymnasium
#%pip install torch

In [2]:
import logging as _logging_
_logging_.basicConfig(level='DEBUG')
#from energyplus.ooep.addons.logging import LogProvider

import energyplus.ooep as _ooep_

from energyplus.ooep.addons.display import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = _ooep_.Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

await simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        model=Model().open(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=Weather().open(_epds_.weathers / 'USA_FL_Tampa.Intl.AP.722110_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(
        report=Report().open('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-6' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:22>>

Exception ignored on calling ctypes callback function: <function EventManager._ep_callback_setters.<locals>.<dictcomp>.<lambda>.<locals>.<lambda> at 0x7fd5869a3920>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 76, in <lambda>
    state, lambda _: trigger(StateEvent(specs=specs))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 53, in trigger
    raise e
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 47, in trigger
    self.trigger(*args, **kwargs)
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/events.py", line 43, in trigger
    return self._handlers[event.specs](event, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/containers.

exception 
end beeebab07fe8438ea48d0c55703a336c {'temperature': array([26.031557], dtype=float32)}
start None


Exception ignored on calling ctypes callback function: <function EventManager._ep_callback_setters.<locals>.<dictcomp>.<lambda>.<locals>.<lambda> at 0x7fd460837a60>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 76, in <lambda>
    state, lambda _: trigger(StateEvent(specs=specs))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 53, in trigger
    raise e
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 47, in trigger
    self.trigger(*args, **kwargs)
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/events.py", line 43, in trigger
    return self._handlers[event.specs](event, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/containers.

...

exception 


Exception ignored on calling ctypes callback function: <function EventManager._ep_callback_setters.<locals>.<dictcomp>.<lambda>.<locals>.<lambda> at 0x7fd460837a60>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 76, in <lambda>
    'after_new_environment_warmup_complete': api.callback_after_new_environment_warmup_complete,
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 53, in trigger
    return {
            ^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 47, in trigger
    except Exception as e:
        ^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/events.py", line 43, in trigger
    return self._handlers[event.specs](event, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

exception 


Exception ignored on calling ctypes callback function: <function EventManager._ep_callback_setters.<locals>.<dictcomp>.<lambda>.<locals>.<lambda> at 0x7fd460837a60>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 76, in <lambda>
    'after_new_environment_warmup_complete': api.callback_after_new_environment_warmup_complete,
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 53, in trigger
    return {
            ^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 47, in trigger
    except Exception as e:
        ^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/events.py", line 43, in trigger
    return self._handlers[event.specs](event, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

exception 


In [3]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl.gymnasium.spaces import VariableBox
from energyplus.ooep.addons.rl.ray import SimulatorEnv

from energyplus.ooep.components.variables import (
    Actuator,
    OutputVariable,
)

from ray.rllib.algorithms.ppo import PPOConfig


config = (
    PPOConfig.from_dict({        
        'create_env_on_local_worker': True
    })
    .resources(num_gpus=1)
    .environment(
        SimulatorEnv, 
        env_config=SimulatorEnv.Config(
            action_space=_gymnasium_.spaces.Dict({
                'thermostat': VariableBox(
                    low=15., high=16.,
                    dtype=_numpy_.float32,
                ).bind(Actuator.Specs(
                    type='Zone Temperature Control',
                    control_type='Heating Setpoint',
                    key='CORE_MID',
                ))
            }),    
            observation_space=_gymnasium_.spaces.Dict({
                'temperature': VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                ).bind(OutputVariable.Specs(
                    type='People Air Temperature',
                    key='CORE_MID',
                )),
            }),
            reward_function=lambda __x: 1,
            event_keys=[
                'begin_zone_timestep_after_init_heat_balance',
            ],
            simulator_factory=lambda simulator=simulator: simulator,
        )
    )
    .rollouts(
        #num_rollout_workers=10,
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .framework("torch")
    .training(model={"fcnet_hiddens": [64, 64]},
             lr=0.0001)
    .evaluation(
        #evaluation_interval=1,
        #evaluation_num_workers=0
    )
)


algo = config.build(use_copy=True)

/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:500: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
2024-04-02 00:07:27,976	WARNING unified.py:56 -- Could not instantiate JsonLogger: cannot pickle '_hashlib.HMAC' objec

In [4]:
# force simulator to rerun to make the newly requested variables available
simulator.stop()
# start training
for _ in range(10):
    print(algo.train())

end None None
start None
end 18ceccc4d5e84cd5a391ca85faeb2746 {'temperature': array([25.865873], dtype=float32)}
start None
end 89f964ad29ff4acf97703118ea8579ce {'temperature': array([25.865873], dtype=float32)}
start None
end 60a3a5c17423487885aa7355893c47e1 {'temperature': array([25.865873], dtype=float32)}
start None
end 03ebb4cd493c4833aaf8da7d63d49a51 {'temperature': array([25.865873], dtype=float32)}
start None
end b9f1c21a1d8f4b00a0f1ccf06e26a06e {'temperature': array([25.865873], dtype=float32)}
start None


2024-04-02 00:07:36,667	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.00010000000000000003, 'total_loss': 9.938017429844026, 'policy_loss': -0.06221994379515289, 'vf_loss': 10.0, 'vf_explained_var': 0.746683862901503, 'kl': 0.001186933380959652, 'entropy': 1.415470016259019, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_epis

In [5]:
algo.evaluate()

{'evaluation': {'sampler_results': {'episode_reward_max': 14501.0,
   'episode_reward_min': 14501.0,
   'episode_reward_mean': 14501.0,
   'episode_len_mean': 14502.0,
   'episode_media': {},
   'episodes_this_iter': 3,
   'policy_reward_min': {},
   'policy_reward_max': {},
   'policy_reward_mean': {},
   'custom_metrics': {},
   'hist_stats': {'episode_reward': [14501.0, 14501.0, 14501.0],
    'episode_lengths': [14502, 14502, 14502]},
   'sampler_perf': {'mean_raw_obs_processing_ms': 0.10984076191996782,
    'mean_inference_ms': 0.9744792563514203,
    'mean_action_processing_ms': 0.08304601633122297,
    'mean_env_wait_ms': 1.69950406724142,
    'mean_env_render_ms': 0.0},
   'num_faulty_episodes': 0,
   'connector_metrics': {}},
  'episode_reward_max': 14501.0,
  'episode_reward_min': 14501.0,
  'episode_reward_mean': 14501.0,
  'episode_len_mean': 14502.0,
  'episode_media': {},
  'episodes_this_iter': 3,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean